# 1) Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np 
import datetime as dt
import seaborn as sns
import copy
import matplotlib.pyplot as plt

# 2) Importing + Concatenating Data From 3 months (12 weeks) Jul 2021 - Sep 2021

In [ ]:
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [210703, 210710, 210717, 210731, 210807, 210814, 210821, 210828, 210904, 210911, 210918, 210925]
df_raw = get_data(week_nums)
df_raw

In [ ]:
df_raw.DATE.value_counts().sort_index() 

# 3) Converting date and time to datetime objects

In [ ]:
df_raw['DATE_TIME'] = pd.to_datetime(df_raw.DATE + ' ' + df_raw.TIME, format='%m/%d/%Y  %H:%M:%S')
df_raw

# 4) Remove leading or trailing spaces from column names

In [ ]:
df_raw.columns = [column.strip() for column in df_raw.columns]
df_raw.columns

# 5) Dropping duplicate index entries

In [ ]:
df_raw = df_raw.loc[~df_raw.index.duplicated(), :]
df_raw.index.is_unique

# 6) Retrieving relevant columns for further handling

In [ ]:
df = df_raw.copy(deep=True)
df.drop(['C/A', 'UNIT', 'SCP', 'LINENAME', 'DIVISION', 'DESC'], axis=1, inplace= True)

# 7) Shifting values up to find the daily difference in **enteries** 
By subtracting total entries of each previous day from the next (current) day 

In [ ]:
df[["PREV_DATE", "PREV_ENTRIES"]] = (df.groupby(["STATION"])["DATE", "ENTRIES"].apply(lambda grp: grp.shift(-1)))

# 8) Subtracting values of **entries**, removing values associated with the turnstiles resetting + handling negative values after subtraction 
Negative values are obtained if the previous day has more entries than the next (current)

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        
        counter = -counter
    if counter > max_counter:
        
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        
        return 0
    return counter

df["DAILY_ENTRIES"] = df.apply(get_daily_counts, axis=1, max_counter=1000000)

# 9) Shifting values to find the daily difference in **exits**
By subtracting total exits of each previous day from the next (current) day

In [ ]:
df[["PREV_DATE", "PREV_EXITS"]] = (df.groupby(["STATION"])["DATE", "EXITS"].apply(lambda grp: grp.shift(-1)));

# 10) Subtracting values of **exits** and removing values associated with the turnstiles resetting + handling negative values
Negative values are obtained if the previous day has more exits than the next (current)

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    if counter < 0:
        
        counter = -counter
    if counter > max_counter:
        
        print(row["EXITS"], row["PREV_EXITS"])
        counter = min(row["EXITS"], row["PREV_EXITS"])
    if counter > max_counter:

        return 0
    return counter
df["DAILY_EXITS"] = df.apply(get_daily_counts, axis=1, max_counter=1000000)

# 11) Dropping the last NaN row + unneeded columns (PREV_DATE, PREV_ENTRIES, PREV_EXITS)

In [ ]:
df.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
df.drop(["PREV_DATE", "PREV_ENTRIES", "PREV_EXITS"],axis = 1 ,inplace = True)

# 12) Get total activity for passengers

In [ ]:
df["TOTAL_ACTIVITY"] = df["DAILY_ENTRIES"] + df["DAILY_EXITS"]

# 13) Deal with outliers in the dataset using boxplot approach

In [ ]:
sns.boxplot(df.TOTAL_ACTIVITY)

In [ ]:
q1 = np.quantile(df.TOTAL_ACTIVITY, 0.25)
q3 = np.quantile(df.TOTAL_ACTIVITY, 0.75)
IQR = q3 - q1
med = np.median(df.TOTAL_ACTIVITY)
upper_bound = q3+(1.5*IQR)
lower_bound = q1-(1.5*IQR)
print(upper_bound,lower_bound)
df = df[(df["TOTAL_ACTIVITY"] < upper_bound) & (df["TOTAL_ACTIVITY"] > lower_bound)]
df.TOTAL_ACTIVITY.max()

In [ ]:
sns.boxplot(df.TOTAL_ACTIVITY)

# 14) Create grouped dataframe to extract necessary information

In [ ]:
df_grouped = (df.groupby(["STATION","DATE_TIME","DATE"],as_index=False).TOTAL_ACTIVITY.sum())

# 15) Get the top 3 stations in terms of activity in the span of Jul 2021 - Sep 2021

In [ ]:
top_3 = df_grouped.groupby(["STATION"],as_index=False).TOTAL_ACTIVITY.sum().sort_values(by = 'TOTAL_ACTIVITY', ascending = False).head(3)
top_3.set_index('STATION',inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(40, 20))
top_3.sort_values(by = 'TOTAL_ACTIVITY').plot(kind='barh',color=(0.2, 0.2, 0.2, 0.2),edgecolor='black',ax=ax)

ax.set_title('Top 3 Busiest Stations by Total Activity', fontsize=50)
ax.set_xlabel('Total Activity', fontsize=40)
ax.set_ylabel('Station', fontsize=40)

for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(35)
ax.legend().set_visible(False)

# 16) Display total activity for each station per date + time

In [ ]:
df_grouped[(df_grouped["STATION"] == '34 ST-PENN STA') | (df_grouped["STATION"] == '23 ST') | (df_grouped["STATION"] == 'FULTON ST')].groupby(["STATION","DATE_TIME"], as_index=False).TOTAL_ACTIVITY.sum()
top_station = df_grouped[(df_grouped["STATION"] == '34 ST-PENN STA')]

In [ ]:
top_station_stats = top_station[['DATE','TOTAL_ACTIVITY']]
top_station_stats['DAY']= pd.to_datetime(top_station_stats['DATE']).dt.day_name()

In [ ]:
weekdays= ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday',]
top_station_stats =top_station_stats.groupby(['DAY'],as_index=False) ['TOTAL_ACTIVITY'].sum()
top_station_stats.index = pd.CategoricalIndex(top_station_stats['DAY'],categories=weekdays,ordered = True )
top_station_stats= top_station_stats.sort_index().reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8), facecolor='white')
ax.bar(top_station_stats['DAY'], top_station_stats['TOTAL_ACTIVITY'],color = ['blue', 'blue', 'blue', 'blue', 'blue','lightblue','lightblue'])
ax.set_xticks([x for x in range(0,7)])
ax.set_xticklabels(top_station_stats['DAY']);
ax.set_title('Total Activity by weekday for 34 ST-PENN STATION', fontsize=18)
ax.set_xlabel('Weekday', fontsize=15)
ax.set_ylabel('Total Activity', fontsize=15);

In [ ]:
time_interval = '4H'
top_1 = df[df['STATION'] == '34 ST-PENN STA']
top_1 = top_1[['STATION', 'DATE_TIME', 'DATE','TIME','TOTAL_ACTIVITY']]
top_1_grouped = top_1.groupby(['STATION', 'DATE_TIME'], as_index = False)['TOTAL_ACTIVITY'].sum()
top_1_grouped = top_1_grouped.set_index('DATE_TIME').groupby(['STATION'])
rounded_grouped = top_1_grouped.resample(time_interval, convention = 'end').sum()
rounded_grouped = rounded_grouped.reset_index()

In [ ]:
rounded_grouped['DAY'] = rounded_grouped['DATE_TIME'].dt.dayofweek
rounded_grouped['TIME'] = rounded_grouped['DATE_TIME'].dt.time
rounded_grouped = rounded_grouped.set_index(['DAY','TIME']).TOTAL_ACTIVITY.unstack(0)
weekdays = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
rounded_grouped.columns = [weekdays[c] for c in rounded_grouped.columns]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set(title='TOTAL ACTIVITY for 34 ST-PENN STATION' , xlabel='', ylabel='Time')
sns.heatmap(rounded_grouped,ax=ax, cmap='Blues')
plt.show()